In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# ==============================================================================
# 1. 配置层
# ==============================================================================
class Config:
    # 输入文件路径
    INPUT_FILE = Path(r'E:\A智网\月度电力需求预测报告\0_数据\异常用户电量数据.xlsx')
    SHEET_NAME = 'huizong'
    
    # 输出文件路径 (与输入文件在同一目录)
    OUTPUT_FILE = INPUT_FILE.parent / '异常用户电量分析结果.xlsx'

    # 定义时间段 (格式: YYYYMMDD 整数或字符串均可，代码会处理)
    # 本期 (2026年1月)
    PERIOD_CURRENT_START = 20260103
    PERIOD_CURRENT_END   = 20260115
    
    # 上年同期 (2025年1月)
    PERIOD_LAST_YEAR_START = 20250103
    PERIOD_LAST_YEAR_END   = 20250115
    
    # 上期 (2025年12月) - 用于环比
    PERIOD_LAST_MONTH_START = 20251203
    PERIOD_LAST_MONTH_END   = 20251215

    # 单位转换 (千瓦时 -> 万千瓦时)
    UNIT_DIVISOR = 10000

# ==============================================================================
# 2. 核心逻辑
# ==============================================================================

def calculate_period_avg(df, start_date, end_date):
    """
    计算指定时间段内每个用户的日均电量
    返回: Series (index=[cust_no, cust_name], value=日均电量)
    """
    # 筛选日期范围
    mask = (df['ds_int'] >= start_date) & (df['ds_int'] <= end_date)
    df_period = df[mask].copy()
    
    if df_period.empty:
        return None
    
    # 计算日均：先按用户分组，计算总电量和天数，然后相除
    # 使用 mean() 直接计算日均是最简单的，前提是数据没有缺失天数。
    # 如果数据可能缺失天数且你希望除以“理论天数”，则需要用 sum() / 理论天数。
    # 这里假设数据是存在的即计算，使用 mean()。
    
    # 分组计算日均
    grp = df_period.groupby(['cust_no', 'cust_name'])['total_eq'].mean()
    
    # 单位转换
    return grp / Config.UNIT_DIVISOR

def calculate_growth_rate(current, base):
    """计算增长率"""
    if pd.isna(current) or pd.isna(base) or base == 0:
        return np.nan
    return (current - base) / abs(base)

def main():
    print("开始处理异常用户电量数据...")
    
    if not Config.INPUT_FILE.exists():
        print(f"[错误] 文件不存在: {Config.INPUT_FILE}")
        return

    try:
        # 1. 读取数据
        # dtype={'cust_no': str} 确保编号不被读成科学计数法
        df = pd.read_excel(Config.INPUT_FILE, sheet_name=Config.SHEET_NAME, dtype={'cust_no': str})
        
        # 数据清洗
        # 将 ds 转为整数以便比较 (假设源数据是 20250103 这种格式)
        # 如果源数据是 datetime 对象，则需要转换。这里做一个兼容处理。
        if pd.api.types.is_datetime64_any_dtype(df['ds']):
            df['ds_int'] = df['ds'].dt.strftime('%Y%m%d').astype(int)
        else:
            # 尝试转为数字，非数字转NaN然后填充0 (虽然日期不应该为0)
            df['ds_int'] = pd.to_numeric(df['ds'], errors='coerce').fillna(0).astype(int)

        df['total_eq'] = pd.to_numeric(df['total_eq'], errors='coerce').fillna(0)
        
        # 去除用户名称和编号的空格
        df['cust_no'] = df['cust_no'].str.strip()
        df['cust_name'] = df['cust_name'].str.strip()

        print("数据读取与清洗完成。")

        # 2. 计算各时期日均电量
        print("正在计算各时期日均电量...")
        
        # 本期 (2026.1)
        avg_current = calculate_period_avg(df, Config.PERIOD_CURRENT_START, Config.PERIOD_CURRENT_END)
        
        # 上年同期 (2025.1)
        avg_last_year = calculate_period_avg(df, Config.PERIOD_LAST_YEAR_START, Config.PERIOD_LAST_YEAR_END)
        
        # 上期 (2025.12)
        avg_last_month = calculate_period_avg(df, Config.PERIOD_LAST_MONTH_START, Config.PERIOD_LAST_MONTH_END)

        # 3. 汇总与合并
        # 获取所有出现过的用户 (并集)
        all_users = set()
        if avg_current is not None: all_users.update(avg_current.index)
        if avg_last_year is not None: all_users.update(avg_last_year.index)
        if avg_last_month is not None: all_users.update(avg_last_month.index)
        
        # 创建结果 DataFrame
        result_df = pd.DataFrame(index=pd.MultiIndex.from_tuples(all_users, names=['cust_no', 'cust_name']))
        
        # 填入数据
        result_df['本期日均'] = avg_current if avg_current is not None else np.nan
        result_df['上年同期日均'] = avg_last_year if avg_last_year is not None else np.nan
        result_df['上期日均'] = avg_last_month if avg_last_month is not None else np.nan
        
        # 4. 计算同比和环比
        print("正在计算同比和环比...")
        result_df['同比'] = result_df.apply(lambda x: calculate_growth_rate(x['本期日均'], x['上年同期日均']), axis=1)
        result_df['环比'] = result_df.apply(lambda x: calculate_growth_rate(x['本期日均'], x['上期日均']), axis=1)

        # 5. 格式化与输出
        result_df = result_df.reset_index()
        
        # 重命名列以符合您的要求
        final_columns = {
            'cust_no': '用户编号',
            'cust_name': '用户名称',
            '本期日均': '1.3-1.15日均电量',
            '上年同期日均': '上年同期日均电量',
            '上期日均': '12月同期日均电量'
        }
        result_df = result_df.rename(columns=final_columns)
        
        # 调整列顺序
        cols_order = [
            '用户编号', '用户名称', 
            '1.3-1.15日均电量', 
            '上年同期日均电量', '同比',
            '12月同期日均电量', '环比'
        ]
        result_df = result_df[cols_order]

        # 格式化百分比和数字
        # 注意：写入Excel时保留数字格式以便后续计算，这里只做简单的四舍五入
        num_cols = ['1.3-1.15日均电量', '上年同期日均电量', '12月同期日均电量']
        result_df[num_cols] = result_df[num_cols].round(4)
        
        # 写入 Excel
        print(f"正在保存结果到: {Config.OUTPUT_FILE}")
        with pd.ExcelWriter(Config.OUTPUT_FILE, engine='openpyxl') as writer:
            result_df.to_excel(writer, index=False, sheet_name='分析结果')
            
            # 设置百分比格式 (使用 xlsxwriter 或 openpyxl 的样式设置比较繁琐，这里直接在 pandas 中转为字符串百分比，方便查看)
            # 如果您希望保留数字格式，可以注释掉下面这段，在 Excel 中手动设置 % 格式
            result_df_formatted = result_df.copy()
            result_df_formatted['同比'] = result_df_formatted['同比'].apply(lambda x: f"{x:.2%}" if pd.notna(x) else "-")
            result_df_formatted['环比'] = result_df_formatted['环比'].apply(lambda x: f"{x:.2%}" if pd.notna(x) else "-")
            result_df_formatted.to_excel(writer, index=False, sheet_name='分析结果(带格式)')

        print("任务完成！")

    except Exception as e:
        print(f"[错误] 执行过程中发生异常: {e}")
        import traceback
        traceback.print_exc()

if __name__ == '__main__':
    main()

开始处理异常用户电量数据...
数据读取与清洗完成。
正在计算各时期日均电量...
正在计算同比和环比...
正在保存结果到: E:\A智网\月度电力需求预测报告\0_数据\异常用户电量分析结果.xlsx
任务完成！


In [2]:
import pandas as pd
from pathlib import Path
import os

# ==============================================================================
# 1. 配置层
# ==============================================================================
class Config:
    # 1. 刚刚生成的分析结果文件 (包含日均、同比、环比)
    FILE_METRICS = Path(r'E:\A智网\月度电力需求预测报告\0_数据\异常用户电量分析结果.xlsx')
    
    # 2. 需要匹配的用户信息文件 (包含行业、送电日期)
    FILE_USER_INFO = Path(r'E:\A智网\月度电力需求预测报告\0_数据\2026.1地市摸排用户.xlsx')
    SHEET_USER_INFO = '电量处理'
    
    # 3. 输出设置
    OUTPUT_SHEET_NAME = '匹配后电量分析'
    OUTPUT_UNMATCHED_SHEET = '未匹配用户清单' # 新增：未匹配用户的Sheet名

# ==============================================================================
# 2. 主流程
# ==============================================================================
def main():
    print("开始执行数据匹配任务...")

    if not Config.FILE_METRICS.exists():
        print(f"[错误] 找不到分析结果文件: {Config.FILE_METRICS}")
        return
    if not Config.FILE_USER_INFO.exists():
        print(f"[错误] 找不到用户信息文件: {Config.FILE_USER_INFO}")
        return

    try:
        # --- 1. 读取数据 ---
        print("正在读取数据...")
        # 强制编号为字符串，确保能匹配上
        df_info = pd.read_excel(Config.FILE_USER_INFO, sheet_name=Config.SHEET_USER_INFO, dtype={'用户编号': str})
        df_metrics = pd.read_excel(Config.FILE_METRICS, sheet_name='分析结果', dtype={'用户编号': str})

        # --- 2. 数据清洗 ---
        df_info['用户编号'] = df_info['用户编号'].astype(str).str.strip()
        df_metrics['用户编号'] = df_metrics['用户编号'].astype(str).str.strip()

        # --- 3. 数据合并 (带指示器) ---
        print("正在根据【用户编号】进行匹配...")
        
        # indicator=True 会增加一列 '_merge'，显示匹配状态
        df_merged = pd.merge(
            df_info, 
            df_metrics, 
            on='用户编号', 
            how='left', 
            suffixes=('', '_分析表'),
            indicator=True 
        )

        # --- 4. 提取未匹配用户 ---
        # 筛选出 _merge 为 'left_only' 的行，这些就是摸排表有，但分析表没有的用户
        df_unmatched = df_merged[df_merged['_merge'] == 'left_only'].copy()
        
        # 只保留摸排表的基础信息列
        unmatched_cols = ['用户编号', '用户名称', '行业名称', '送电日期']
        # 防止列名不存在报错，取交集
        valid_unmatched_cols = [c for c in unmatched_cols if c in df_unmatched.columns]
        df_unmatched = df_unmatched[valid_unmatched_cols]

        print(f"  -> 发现 {len(df_unmatched)} 个用户未匹配到电量数据。")

        # --- 5. 处理匹配后的主表 ---
        # 定义列映射 (Key必须和分析结果表的列名一致)
        col_mapping = {
            '2026.1.3-1.15日均电量(万kWh)': '1.3-1.15日均电量',
            '上年同期日均电量(万kWh)': '上年同期日均电量',
            '同比': '同比',
            '2025.12.3-12.15日均电量(万kWh)': '12月同期日均电量',
            '环比': '环比'
        }
        
        df_merged.rename(columns=col_mapping, inplace=True)
        
        final_cols = [
            '用户编号', '用户名称', '行业名称', '送电日期', 
            '1.3-1.15日均电量', '上年同期日均电量', '同比', '12月同期日均电量', '环比'
        ]
        
        # 提取最终需要的列
        df_final = df_merged.reindex(columns=final_cols)

        # --- 6. 写入 Excel ---
        print(f"正在写入文件: {Config.FILE_USER_INFO.name} ...")
        
        with pd.ExcelWriter(Config.FILE_USER_INFO, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            # 写入主结果
            df_final.to_excel(writer, sheet_name=Config.OUTPUT_SHEET_NAME, index=False)
            print(f"  -> 已写入 Sheet: {Config.OUTPUT_SHEET_NAME}")
            
            # 写入未匹配清单
            if not df_unmatched.empty:
                df_unmatched.to_excel(writer, sheet_name=Config.OUTPUT_UNMATCHED_SHEET, index=False)
                print(f"  -> 已写入 Sheet: {Config.OUTPUT_UNMATCHED_SHEET}")
            else:
                print("  -> 所有用户均已匹配，无需生成未匹配清单。")
            
        print(f"\n任务完成！请打开文件查看结果。")

    except Exception as e:
        print(f"[错误] 执行过程中发生异常: {e}")
        import traceback
        traceback.print_exc()

if __name__ == '__main__':
    main()

开始执行数据匹配任务...
正在读取数据...
正在根据【用户编号】进行匹配...
  -> 发现 1 个用户未匹配到电量数据。
正在写入文件: 2026.1地市摸排用户.xlsx ...
  -> 已写入 Sheet: 匹配后电量分析
  -> 已写入 Sheet: 未匹配用户清单

任务完成！请打开文件查看结果。


In [7]:
import pandas as pd
from pathlib import Path
import os

# ==============================================================================
# 1. 配置层
# ==============================================================================
class Config:
    # 1. 分析结果文件 (数据源)
    FILE_METRICS = Path(r'E:\A智网\月度电力需求预测报告\0_数据\异常用户电量分析结果.xlsx')
    
    # 2. 摸排用户文件 (目标文件，既读也写)
    FILE_USER_INFO = Path(r'E:\A智网\月度电力需求预测报告\0_数据\2026.1地市摸排用户.xlsx')
    SHEET_TARGET = '电量处理' 
    
    # 3. 未匹配清单 Sheet 名
    SHEET_UNMATCHED = '未匹配用户清单'

# ==============================================================================
# 2. 主流程
# ==============================================================================
def main():
    print("开始执行【双重匹配】数据合并任务...")

    # --- 1. 检查文件 ---
    if not Config.FILE_METRICS.exists():
        print(f"[错误] 找不到分析结果文件: {Config.FILE_METRICS}")
        return
    if not Config.FILE_USER_INFO.exists():
        print(f"[错误] 找不到用户信息文件: {Config.FILE_USER_INFO}")
        return

    try:
        # --- 2. 读取数据 ---
        print("正在读取数据...")
        # 读取主表 (保留原表所有数据)
        df_main = pd.read_excel(Config.FILE_USER_INFO, sheet_name=Config.SHEET_TARGET, dtype={'用户编号': str})
        
        # 读取指标表
        df_metrics = pd.read_excel(Config.FILE_METRICS, sheet_name='分析结果', dtype={'用户编号': str})

        # --- 3. 数据清洗 (关键步骤) ---
        print("正在清洗编号和名称...")
        # 同时清洗两个关键字段，去除首尾空格，确保精确匹配
        # 主表清洗
        df_main['用户编号'] = df_main['用户编号'].astype(str).str.strip()
        df_main['用户名称'] = df_main['用户名称'].astype(str).str.strip()
        
        # 指标表清洗
        df_metrics['用户编号'] = df_metrics['用户编号'].astype(str).str.strip()
        df_metrics['用户名称'] = df_metrics['用户名称'].astype(str).str.strip()

        # --- 4. 准备指标列 (重命名) ---
        col_mapping = {
            '1.3-1.15日均电量': '1.3-1.15日均电量',
            '上年同期日均电量': '上年同期日均电量',
            '同比': '同比',
            '12月同期日均电量': '12月同期日均电量',
            '环比': '环比'
        }
        
        # 提取需要的列：【用户编号】+【用户名称】+【5个指标】
        # 注意：这里必须包含 '用户名称'，因为它是匹配键之一
        cols_to_extract = ['用户编号', '用户名称'] + list(col_mapping.keys())
        
        # 过滤掉分析表中可能不存在的列
        cols_to_extract = [c for c in cols_to_extract if c in df_metrics.columns]
        
        # 提取并重命名
        df_metrics_subset = df_metrics[cols_to_extract].rename(columns=col_mapping)

        # --- 5. 清理主表中可能已存在的旧指标列 ---
        target_cols = list(col_mapping.values())
        df_main = df_main.drop(columns=[c for c in target_cols if c in df_main.columns], errors='ignore')

        # --- 6. 数据合并 (双重匹配) ---
        print("正在根据【用户编号 + 用户名称】进行严格匹配...")
        
        # 使用 left join，以主表为准
        # on 参数传入列表，表示必须两个字段都相等才算匹配成功
        df_final = pd.merge(
            df_main, 
            df_metrics_subset, 
            on=['用户编号', '用户名称'], 
            how='left', 
            indicator=True
        )

        # --- 7. 提取未匹配用户 ---
        # 这里的未匹配意味着：编号不对，或者名称不对，或者两者都不对
        df_unmatched = df_final[df_final['_merge'] == 'left_only'].copy()
        
        # 仅保留基础信息用于核对
        base_cols = ['用户编号', '用户名称', '行业名称']
        valid_base_cols = [c for c in base_cols if c in df_unmatched.columns]
        df_unmatched = df_unmatched[valid_base_cols]

        # --- 8. 清理辅助列 ---
        df_final = df_final.drop(columns=['_merge'])

        # --- 9. 写入 Excel ---
        print(f"正在更新文件: {Config.FILE_USER_INFO.name} ...")
        print("请注意：请确保该 Excel 文件已关闭！")
        
        with pd.ExcelWriter(Config.FILE_USER_INFO, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            # 1. 写入更新后的主表
            df_final.to_excel(writer, sheet_name=Config.SHEET_TARGET, index=False)
            print(f"  -> 已更新 Sheet: {Config.SHEET_TARGET} (双重匹配完成)")
            
            # 2. 写入未匹配清单
            if not df_unmatched.empty:
                df_unmatched.to_excel(writer, sheet_name=Config.SHEET_UNMATCHED, index=False)
                print(f"  -> 已生成 Sheet: {Config.SHEET_UNMATCHED} (共 {len(df_unmatched)} 个未匹配)")
                print("     (提示：未匹配可能是因为两张表中的用户名称写法不完全一致)")
            else:
                print("  -> 完美！所有用户均已成功匹配。")
            
        print(f"\n任务完成！")

    except PermissionError:
        print("\n[错误] 无法写入文件。请关闭 Excel 文件后重试！")
    except Exception as e:
        print(f"\n[错误] 执行过程中发生异常: {e}")
        import traceback
        traceback.print_exc()

if __name__ == '__main__':
    main()

开始执行【双重匹配】数据合并任务...
正在读取数据...
正在清洗编号和名称...
正在根据【用户编号 + 用户名称】进行严格匹配...
正在更新文件: 2026.1地市摸排用户.xlsx ...
请注意：请确保该 Excel 文件已关闭！
  -> 已更新 Sheet: 电量处理 (双重匹配完成)
  -> 已生成 Sheet: 未匹配用户清单 (共 1 个未匹配)
     (提示：未匹配可能是因为两张表中的用户名称写法不完全一致)

任务完成！
